## Given a data, first identify whether it's from/to airport and then use algorithm

In [2]:
import pandas as pd
import numpy as np
import sklearn.metrics as skl
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
%matplotlib inline

/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
## Based on the above, let us create a function to see whether pickup or dropoff is an these three airports


def isAirport(latitude,longitude):
        JFK={'min_lng':-73.8352,
                  'min_lat':40.6195,
                  'max_lng':-73.7401, 
                  'max_lat':40.6659
            }
        
        EWR={'min_lng':-74.1925,
                  'min_lat':40.6700, 
                  'max_lng':-74.1531, 
                  'max_lat':40.7081

            }
        
        LA={'min_lng':-73.8895, 
                  'min_lat':40.7664, 
                  'max_lng':-73.8550, 
                  'max_lat':40.7931
            }
        
#         MANH={'min_lng':-74.0479,
#                   'min_lat':40.6829,
#                   'max_lng':-73.9067,
#                   'max_lat':40.8820
#              }
    
        isJFK = False;
        isEWR = False;
        isLA = False;
#         isMANH = False;
        
        if latitude>=JFK['min_lat'] and latitude<=JFK['max_lat']:
            if longitude>=JFK['min_lng'] and longitude<=JFK['max_lng']:
                isJFK = True;

        if latitude>=EWR['min_lat'] and latitude<=EWR['max_lat']:
            if longitude>=EWR['min_lng'] and longitude<=EWR['max_lng']:
                isEWR = True;

        if latitude>=LA['min_lat'] and latitude<=LA['max_lat']:
            if longitude>=LA['min_lng'] and longitude<=LA['max_lng']:
                isLA = True;

#         if latitude>=MANH['min_lat'] and latitude<=MANH['max_lat']:
#             if longitude>=MANH['min_lng'] and longitude<=MANH['max_lng']:
#                 isMANH = True;

            
        #isArea = isJFK or isEWR or isLA or isMANH
        isAirport = isJFK or isEWR or isLA
        return int(isAirport)
        
    


# nyc_airports={
#     'JFK':{'min_lng':-73.8352,
#            'min_lat':40.6195,
#            'max_lng':-73.7401, 
#            'max_lat':40.6659
#           },
              
#     'EWR':{'min_lng':-74.1925,
#            'min_lat':40.6700, 
#            'max_lng':-74.1531, 
#            'max_lat':40.7081

#         },
#     'LaGuardia':{'min_lng':-73.8895, 
#                  'min_lat':40.7664, 
#                  'max_lng':-73.8550, 
#                  'max_lat':40.7931
        
#     }
    
# }

In [4]:
data = pd.read_csv("data/cleaned_train_with_geo_distance.csv")

In [16]:
airport_data = data[data['isAirport']==1]
airport_data = airport_data.copy()

airport_data.drop(['isAirport'],axis=1,inplace=True)

In [21]:
Airport_features = ['year', 'hour', 'distance', 'passenger_count']
X = airport_data[Airport_features].values

A_X = np.concatenate( ( np.ones((X.shape[0], 1)), X ), axis = 1 )
A_Y = airport_data['fare_amount'].values

In [39]:
d = airport_data.copy()
l = d.shape[0]
train = d.iloc[:200]
test = d.iloc[200:]

In [40]:
X = train.loc[:, Airport_features].values
y = train.iloc[:, 0].values

X_test_dataset = test.loc[:, Airport_features].values

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [41]:
rand_forest_regressor = RandomForestRegressor()
rand_forest_regressor.fit(X_train, y_train)

y_rand_forest_predict = rand_forest_regressor.predict(X_test)
random_forest_model_error = math.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(y_rand_forest_predict)))
print(random_forest_model_error)

1.71728795044e+24


In [42]:
XGB_model = XGBRegressor(learning_rate=0.3, max_depth=6, n_estimators=500)
XGB_model.fit(X_train, y_train)
y_XGB_predict = XGB_model.predict(X_test)

XGB_model_error = math.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(y_XGB_predict)))
print(XGB_model_error)

1.56087141488e+24
